# IMDb 영화 리뷰 감성 분석: 텍스트 전처리 및 벡터화

이 노트북은 IMDb 영화 리뷰 데이터셋을 사용하여 긍정/부정 감성을 분류하는 기본적인 텍스트 분석 과정을 다룹니다. 

특히 텍스트 데이터를 머신러닝 모델이 이해할 수 있는 숫자 형태의 벡터(vector)로 변환하는 **`CountVectorizer`**의 사용법과, 그 과정에서 **텍스트 정제(cleaning)가 왜 중요한지**를 단계별로 알아봅니다.

### 1. 라이브러리 임포트

In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

### 2. 데이터 로드 및 탐색

`scikit-learn`의 `load_files` 함수는 특정 폴더 내의 텍스트 파일들을 로드하는 데 유용합니다. 하위 폴더의 이름(여기서는 `pos`, `neg`)을 자동으로 레이블(label)로 사용합니다.

In [ ]:
# ./data/aclImdb/train 폴더 아래의 텍스트 파일들을 로드합니다.
# 폴더 구조: train/pos, train/neg
try:
    reviews_train = load_files("./data/aclImdb/train")
    print("데이터 로드 성공!")
except FileNotFoundError:
    print("오류: './data/aclImdb/train' 폴더를 찾을 수 없습니다.")
    print("데이터 경로를 확인해주세요.")
    reviews_train = None

In [ ]:
if reviews_train:
    # 로드된 데이터는 Bunch 객체로, 딕셔너리처럼 키로 접근할 수 있습니다.
    text_train_raw = reviews_train.data
    y_train = reviews_train.target
    
    # 첫 번째 리뷰 데이터 확인
    # b'... 형태로, 바이트(byte) 문자열임을 알 수 있습니다.
    # 리뷰 내용 중간에 HTML 줄바꿈 태그인 <br />가 포함되어 있습니다.
    print("첫 번째 리뷰 원본 데이터:")
    print(text_train_raw[0])

### 3. 텍스트 벡터화 (첫 번째 시도 - 정제 전)

`CountVectorizer`는 텍스트 문서를 단어의 빈도수를 나타내는 벡터로 변환합니다. 먼저 아무런 정제 없이 원본 데이터에 바로 적용해 보겠습니다.

In [ ]:
if reviews_train:
    # 정제되지 않은 데이터로 벡터화
    vect_raw = CountVectorizer().fit(text_train_raw)
    feature_names_raw = vect_raw.get_feature_names_out()
    
    print(f"생성된 특성(단어)의 개수: {len(feature_names_raw)}")
    print(f"처음 20개 특성: {feature_names_raw[:20]}")
    print(f"중간 20개 특성: {feature_names_raw[20000:20020]}")
    # 결과에 'br' (from <br />)과 같은 불필요한 특성이 포함된 것을 볼 수 있습니다.

### 4. 텍스트 데이터 정제

분석에 불필요한 HTML 태그(`<br />`)를 제거합니다. 리스트 컴프리헨션을 사용하여 모든 리뷰에 대해 `replace` 함수를 적용합니다.

In [ ]:
if reviews_train:
    # b"<br />"를 b"" (빈 바이트 문자열)로 변경
    text_train_cleaned = [review.replace(b"<br />", b" ") for review in text_train_raw]
    
    print("정제 후 첫 번째 리뷰 데이터:")
    print(text_train_cleaned[0])

### 5. 텍스트 벡터화 (두 번째 시도 - 정제 후)

깨끗하게 정제된 데이터에 `CountVectorizer`를 다시 적용합니다.

In [ ]:
if reviews_train:
    vect = CountVectorizer().fit(text_train_cleaned)
    X_train = vect.transform(text_train_cleaned)
    
    feature_names = vect.get_feature_names_out()
    print(f"정제 후 생성된 특성의 개수: {len(feature_names)}")
    print(f"처음 20개 특성: {feature_names[:20]}")
    print(f"중간 20개 특성: {feature_names[20000:20020]}")
    # 'br'과 같은 불필요한 특성이 사라진 것을 확인할 수 있습니다.
    
    print("
벡터화된 데이터의 형태:", X_train.shape)

### 6. 모델 학습 및 평가

최종적으로 벡터화된 데이터를 사용하여 로지스틱 회귀 모델을 학습시키고 성능을 평가합니다.

**참고**: 여기서는 데이터 전처리 과정에 집중하기 위해 훈련 데이터만 사용하여 학습 및 평가를 진행했습니다. 모델의 일반화 성능을 정확히 측정하려면, 반드시 별도의 **테스트 데이터셋**으로 평가해야 합니다.

In [ ]:
if reviews_train:
    model = LogisticRegression(max_iter=1000) # 반복 횟수를 늘려 모델이 수렴하도록 보장
    model.fit(X_train, y_train)
    
    score = model.score(X_train, y_train)
    print(f"훈련 세트 정확도: {score:.4f}")